In [1]:
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sage_data_client import query

In [2]:
def get_plugins_per_node(jobs):
    for _, j in jobs.iterrows():
        if isinstance(j["science_goal"], dict):
            # print(j["science_goal"]["sub_goals"])
            sub_goals = j["science_goal"]["sub_goals"]
            for sub_goal in sub_goals:
                for plugin in sub_goal["plugins"]:
                    yield pd.Series(data=[j["job_id"], sub_goal["name"], plugin["name"], os.path.basename(plugin["plugin_spec"]["image"])], index=["job", "vsn", "plugin_name", "image"])
    

In [3]:
url = "https://es.sagecontinuum.org/api/v1/jobs/list"
df = pd.read_json(url).T

In [12]:
for p in df[df["job_id"]=="32"].plugins:
    print(p)

[{'name': 'traffic-state-estimator', 'plugin_spec': {'image': 'registry.sagecontinuum.org/seonghapark/traffic-state:0.1.0', 'args': ['-stream', 'rtsp://10.31.81.18:554/0/profile10/media.smp', 'duration', '30', '-roi-area', '40', '-roi-length', '10', '-roi-coordinates', '0.25,0.44 0.44,0.3 0.8,0.44 0.81,0.74', '-loi-coordinates', '0.25,0.44 0.44,0.3', '-sampling-interval', '0'], 'selector': {'resource.gpu': 'true'}}, 'status': {'scheduling': 'Waiting', 'since': '2022-11-09T17:44:27.162011107Z'}, 'goal_id': '6ab1a8dc-c966-4a67-4939-2b29f4412564'}]


In [6]:
data = pd.DataFrame(get_plugins_per_node(df))

In [7]:
grouped_by_image = data.groupby(by=["image"])

In [8]:
data

,job,vsn,plugin_name,image
0,1,V001,imagesampler-neon40,imagesampler:0.3.0
1,1,V001,imagesampler-neon41,imagesampler:0.3.0
2,1,V001,object-counter-neon40,object-counter:0.5.1
3,1,V001,object-counter-neon41,object-counter:0.5.1
4,1,V001,smoke-detector-neon41,wildfire-smoke-detection:0.5.0
...,...,...,...,...
294,9,W02B,imagesampler-bottom,imagesampler:0.3.0
295,9,W02B,cloud-cover-top,cloud-cover:0.1.3
296,9,W02B,cloud-motion-top,cloud-motion:0.22.2.27
297,9,W02B,object-counter-bottom,object-counter:0.5.1


In [9]:
df2 = data.drop_duplicates(subset=["image", "vsn"])

Count number of Waggle nodes that run the edge plugins

In [10]:
df2.groupby("image")["vsn"].count()

image
avian-diversity-monitoring:0.2.4     3
cloud-cover:0.1.3                   32
cloud-motion:0.22.2.27              26
gpu-stress-test:latest               1
imagesampler:0.3.0                  58
object-counter:0.5.1                55
sound-event-detection:0.1.1         27
surface-water-classifier:0.0.5       1
video-sampler:0.2.4                  1
water-depth-estimator:0.0.1          3
wildfire-smoke-detection:0.5.0       7
Name: vsn, dtype: int64

In [11]:
df2.groupby("vsn")["image"].count()

vsn
V001    4
V002    4
V003    5
V004    5
V005    4
V006    5
W014    1
W015    5
W019    5
W01E    2
W021    5
W023    2
W024    6
W026    5
W027    5
W028    4
W029    6
W02B    5
W02C    5
W02D    2
W02E    2
W02F    2
W040    2
W045    5
W046    2
W047    2
W048    2
W049    2
W04A    2
W05F    2
W060    2
W061    2
W062    2
W063    2
W064    2
W069    5
W06A    5
W06D    1
W06F    5
W078    5
W079    5
W07A    5
W07B    5
W07C    5
W07D    5
W07E    5
W07F    5
W080    5
W081    5
W084    6
W087    5
W088    5
W089    2
W08A    2
W08F    2
W090    2
W091    2
W092    2
W093    2
Name: image, dtype: int64